In [1]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import cv2

In [3]:
images_yes = [cv2.imread(file) for file in glob.glob("data/yes/*.*")]
images_no = [cv2.imread(file) for file in glob.glob("data/no/*.*")]
len(images_no)

98

In [4]:
def image_crop(image_path):
    cropped_image = []
    for i in image_path:
        if i is None:
            pass
        else:
            image = cv2.resize(i , (128,128))
        
        #The initial processing of the image
        image = cv2.medianBlur(image, 3)
        image_bw = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        #The declaration of CLAHE 
        #clipLimit -> Threshold for contrast limiting
        clahe = cv2.createCLAHE(clipLimit = 5)
        final_img = clahe.apply(image_bw)
        cropped_image.append(final_img)
    cropped_image = np.array(cropped_image)
    return cropped_image

In [5]:
cropped_images_no = image_crop(images_no)
cropped_images_yes = image_crop(images_yes)

In [6]:
y_yes = np.ones(len(cropped_images_yes), dtype="int8")
y_no = np.zeros(len(cropped_images_no), dtype="int8")

In [7]:
X = np.concatenate((cropped_images_yes, cropped_images_no), axis=0)
y = np.concatenate((y_yes, y_no), axis=0)

d1, d2, d3 = X.shape
X_1 = X.reshape((d1, d2 * d3))

## Functional Model

In [8]:
from sklearn.model_selection import train_test_split

X_train_f, X_test_f, y_train_f, y_test_f = train_test_split(X_1, y, test_size=0.2, random_state=42)

In [9]:
X_train_f.shape

(202, 16384)

In [10]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [11]:
inputs = keras.Input(shape=(128*128))

In [12]:
inputs.shape

TensorShape([None, 16384])

In [13]:
dense = layers.Dense(64, activation="relu")
x = dense(inputs)

In [14]:
x = layers.Dense(64, activation="relu")(x)
outputs = layers.Dense(10)(x)

In [15]:
model = keras.Model(inputs=inputs, outputs=outputs, name="functional_tumor_model")

In [16]:
model.summary()

Model: "functional_tumor_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 16384)]           0         
                                                                 
 dense (Dense)               (None, 64)                1048640   
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dense_2 (Dense)             (None, 10)                650       
                                                                 
Total params: 1,053,450
Trainable params: 1,053,450
Non-trainable params: 0
_________________________________________________________________


In [17]:
#Scaling
X_train = X_train_f / 255
X_test = X_test_f / 255

In [18]:
model.compile(loss=keras.losses.sparse_categorical_crossentropy,
              optimizer=keras.optimizers.RMSprop(), metrics=["accuracy"])

In [19]:
history = model.fit(X_train_f, y_train_f, batch_size=64, epochs=10, validation_split=0.2)

Epoch 1/10
3/3 [==============================] - 1s 137ms/step - loss: 11.5951 - accuracy: 0.2733 - val_loss: 11.0444 - val_accuracy: 0.4146
Epoch 2/10
3/3 [==============================] - 0s 112ms/step - loss: 12.0211 - accuracy: 0.3540 - val_loss: 11.0444 - val_accuracy: 0.4146
Epoch 3/10
3/3 [==============================] - 0s 87ms/step - loss: 12.0211 - accuracy: 0.3540 - val_loss: 11.0444 - val_accuracy: 0.4146
Epoch 4/10
3/3 [==============================] - 0s 68ms/step - loss: 12.0211 - accuracy: 0.3540 - val_loss: 11.0444 - val_accuracy: 0.4146
Epoch 5/10
3/3 [==============================] - 0s 80ms/step - loss: 12.0211 - accuracy: 0.3540 - val_loss: 11.0444 - val_accuracy: 0.4146
Epoch 6/10
3/3 [==============================] - 0s 77ms/step - loss: 12.0211 - accuracy: 0.3540 - val_loss: 11.0444 - val_accuracy: 0.4146
Epoch 7/10
3/3 [==============================] - 0s 70ms/step - loss: 12.0211 - accuracy: 0.3540 - val_loss: 11.0444 - val_accuracy: 0.4146
Epoch 8/10


In [20]:
test_scores = model.evaluate(X_test_f, y_test_f, verbose=2)
print("Test loss:", test_scores[0])
print("Test accuracy:", test_scores[1])

2/2 - 0s - loss: 10.1425 - accuracy: 0.4706 - 51ms/epoch - 25ms/step
Test loss: 10.142546653747559
Test accuracy: 0.47058823704719543


## Subclassing

In [21]:
class Sampling(layers.Layer):
    
#     Uses (z_mean, z_log_var) to sample z, the vector encoding a digit.

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon


class Encoder(layers.Layer):

    def __init__(self, latent_dim=32, intermediate_dim=64, name="encoder", **kwargs):
        super(Encoder, self).__init__(name=name, **kwargs)
        self.dense_proj = layers.Dense(intermediate_dim, activation="relu")
        self.dense_mean = layers.Dense(latent_dim)
        self.dense_log_var = layers.Dense(latent_dim)
        self.sampling = Sampling()

    def call(self, inputs):
        x = self.dense_proj(inputs)
        z_mean = self.dense_mean(x)
        z_log_var = self.dense_log_var(x)
        z = self.sampling((z_mean, z_log_var))
        return z_mean, z_log_var, z


class Decoder(layers.Layer):
#     Converts z, the encoded digit vector, back into a readable digit.

    def __init__(self, original_dim, intermediate_dim=64, name="decoder", **kwargs):
        super(Decoder, self).__init__(name=name, **kwargs)
        self.dense_proj = layers.Dense(intermediate_dim, activation="relu")
        self.dense_output = layers.Dense(original_dim, activation="sigmoid")

    def call(self, inputs):
        x = self.dense_proj(inputs)
        return self.dense_output(x)


class VariationalAutoEncoder(keras.Model):
#     Combines the encoder and decoder into an end-to-end model for training.

    def __init__(
        self,
        original_dim,
        intermediate_dim=64,
        latent_dim=32,
        name="autoencoder",
        **kwargs
    ):
        super(VariationalAutoEncoder, self).__init__(name=name, **kwargs)
        self.original_dim = original_dim
        self.encoder = Encoder(latent_dim=latent_dim, intermediate_dim=intermediate_dim)
        self.decoder = Decoder(original_dim, intermediate_dim=intermediate_dim)

    def call(self, inputs):
        z_mean, z_log_var, z = self.encoder(inputs)
        reconstructed = self.decoder(z)
        # Add KL divergence regularization loss.
        kl_loss = -0.5 * tf.reduce_mean(
            z_log_var - tf.square(z_mean) - tf.exp(z_log_var) + 1
        )
        self.add_loss(kl_loss)
        return reconstructed

In [22]:
X_train_s, X_test_s, y_train_s, y_test_s = train_test_split(X, y, test_size=0.2, random_state=42)

In [30]:
original_dim = 128
vae = VariationalAutoEncoder(original_dim, 64, 32)

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
mse_loss_fn = tf.keras.losses.MeanSquaredError()

loss_metric = tf.keras.metrics.Mean()

train_dataset = tf.data.Dataset.from_tensor_slices(X_train_s)

epochs = 2

# Iterate over epochs.
for epoch in range(epochs):
    print("Start of epoch %d" % (epoch,))

    # Iterate over the batches of the dataset.
    for step, x_batch_train in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            reconstructed = vae(x_batch_train)
            # Compute reconstruction loss
            loss = mse_loss_fn(x_batch_train, reconstructed)
            loss += sum(vae.losses)  # Add KLD regularization loss

        grads = tape.gradient(loss, vae.trainable_weights)
        optimizer.apply_gradients(zip(grads, vae.trainable_weights))

        loss_metric(loss)

        if step % 100 == 0:
            print("step %d: mean loss = %.4f" % (step, loss_metric.result()))

Start of epoch 0
step 0: mean loss = nan
step 100: mean loss = nan
step 200: mean loss = nan
Start of epoch 1
step 0: mean loss = nan
step 100: mean loss = nan
step 200: mean loss = nan


In [36]:
### WHY LOSS IS NAN?

#### OR:

In [32]:
vae_model = VariationalAutoEncoder(128, 64, 32)

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

vae_model.compile(optimizer, loss=tf.keras.losses.MeanSquaredError())
vae_model.fit(X_train_f, y_train_f, epochs=5, batch_size=64)

Epoch 1/5
4/4 [==============================] - 1s 15ms/step - loss: nan
Epoch 2/5
4/4 [==============================] - 0s 19ms/step - loss: nan
Epoch 3/5
4/4 [==============================] - 0s 23ms/step - loss: nan
Epoch 4/5
4/4 [==============================] - 0s 18ms/step - loss: nan
Epoch 5/5
4/4 [==============================] - 0s 16ms/step - loss: nan


In [35]:
test_scores = vae_model.evaluate(X_test_f, y_test_f, verbose=2)
print("Test loss:", test_scores[0])
print("Test accuracy:", test_scores[1])